In [1]:

ini_date = '20/12/22 15:00:00'
end_date = '23/12/22 15:00:00'
target_folder="./time-data/setup/"


time_step=15

binary_sensors=["cubiertos", "fridge", "microwave", "ollas"]

tags=["1","2"]


In [2]:
def repaintXY(x,y):
    return int(x*0.08965-196.41+150), int(-y*0.1078+1077)

def repaintXXYY(x,y):
    return int((x+196.41-150)/0.08965), int((y-1077)/(-0.1078))

print(repaintXY(2280,5135))
print(repaintXY(2860,6730))
print(repaintXXYY(209,351))
#print(repaintXY(5300,6450))
#print(repaintXY(6600,5200))

(157, 523)
(209, 351)
(2848, 6734)


In [3]:
import numpy as np
from pandas import read_csv    

import time

from datetime import datetime

def day_time(ti):
    return int((int)(ti/(60*60*24)))

t0 = (int)(datetime.strptime(ini_date, '%d/%m/%y %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0))

tN = (int)(datetime.strptime(end_date, '%d/%m/%y %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN))



nT=(int)(round((tN-t0-1)/time_step))
print(nT)


def absT(tt):
    return (int)(tt*time_step+t0)

def relT(ti):
    return (int)((ti-t0)/time_step)


20/12/22 15:00:00 Init date is 1671544800 day: 19346
23/12/22 15:00:00 End date is 1671804000 day: 19349
17280


In [4]:
user_location={}
for tag in tags:
    user_location[tag]=[]
    series=read_csv(target_folder+tag+".tsv", sep="\t",parse_dates=True,header=None)
    user_location[tag]=  np.array(series.values[:,[1,2,3,4,5]])  
    print(user_location[tag].shape)
print(user_location["1"])

(17280, 5)
(17280, 5)
[['None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None']
 ...
 ['2028.6040028752798' '6596.97275770177' '2408.723468772463'
  '6786.831092901128' '1']
 ['1870.0145094890609' '6660.8905185847725' '2615.857214599785'
  '6716.993189425442' '1']
 ['1680.1328691776916' '6660.096055971507' '2598.991731657514'
  '6828.048027985753' '1']]


In [5]:
S=np.zeros((len(binary_sensors), nT))
for i,sensor in enumerate(binary_sensors):
    series=read_csv(target_folder+sensor+".tsv", sep="\t",parse_dates=True,header=None)
    S[i]=  np.array(series.values[:,[1]]).flatten()  
    print(S[i].shape)
print(S.shape)
print(np.where( (S[0]==1)))

(17280,)
(17280,)
(17280,)
(17280,)
(4, 17280)
(array([   48,    49,   124,   128,   129,   146,   147,   196,   216,
         218,   304,   328,   330,   333,   334,   335,   342,   343,
         365,   613,   615,   620,   621,   636,   661,   672,   680,
        1603,  1616,  1617,  1635,  1636,  1638,  1786,  1787,  1811,
        1812,  4186,  4187,  4196,  4209,  4236,  4237,  4240,  4246,
        4247,  4356,  4417,  4424,  4425,  5052,  5061,  5104,  5110,
        5114,  5141,  5143,  5156,  5189,  5206,  5207,  5208,  5227,
        5228,  5328,  5418,  5489,  5490,  5546,  5547,  5548,  5634,
        5635,  5637,  5648,  5668,  5669,  5671,  6011,  6232,  6233,
        6239,  6243,  6249,  6258,  6272,  6273,  6283,  6288,  6292,
        6312,  7247,  7255,  7256,  7259,  7262,  7263,  7274,  7275,
        7279,  7280,  7289,  7290,  7292,  7387,  7397,  7399,  9578,
        9777,  9778,  9845, 10738, 10740, 10805, 10842, 10843, 11307,
       11308, 11354, 11380, 11381, 11410, 

In [6]:
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from datetime import datetime

X_train={}
Y_train={}

for i,sensor in enumerate(binary_sensors):
    index_activation=np.where( (S[i]==1))[0]
    print("Sensor ",sensor, "#",i, "activation:",index_activation)
    
    X_train[sensor]=[]
    Y_train[sensor]=[]

    y_true={}
    gt=read_csv(target_folder+"gt-labelling-sensors/"+sensor+".tsv", sep="\t",parse_dates=True, na_values=0)
    gt=gt.fillna(0)
    for tx,tt in enumerate(gt["Ix"]):
        
        if(sum(gt.values[tx][2:])==0):
            maxI=0
        else:
            maxI=np.argmax(gt.values[tx][2:])+1
        y_true[tt]=(maxI)
    print("Sensor ",sensor, "#",i, "gt activation:",gt["Ix"])   
        
    
    for t_activation in gt["Ix"]:
        t_activation=int(t_activation)
        print(t_activation,absT(t_activation))
        t=absT(t_activation)
        #los datos de mi auro label (output) + coordenadas de 1 y 2 (input)
        #print("\t time_index:", t_activation, "LABEL:", y_true[t_activation], "in time", datetime.fromtimestamp(t).strftime("%Y-%m-%d %H:%M:%S"), "user 1:",user_location["1"][t_activation], "user 2:",user_location["2"][t_activation])
        
        user1_coords = [float(coord) if coord != 'None' else 0 for coord in user_location["1"][t_activation][:4]]
        user2_coords = [float(coord) if coord != 'None' else 0 for coord in user_location["2"][t_activation][:4]]
        label = y_true[t_activation]
        
        
        if label == 'None':
            label = 0

        if(int(label)>2):
            continue
        x1, x2, y1, y2 = user1_coords
        x1_2, x2_2, y1_2, y2_2 = user2_coords
        X_train[sensor].append([x1, x2, y1, y2, x1_2, x2_2, y1_2, y2_2])

        Y_train[sensor].append(int(label))
    X_train[sensor] = np.array(X_train[sensor])
    Y_train[sensor] = np.array(Y_train[sensor])




    
    

Sensor  cubiertos # 0 activation: [   48    49   124   128   129   146   147   196   216   218   304   328
   330   333   334   335   342   343   365   613   615   620   621   636
   661   672   680  1603  1616  1617  1635  1636  1638  1786  1787  1811
  1812  4186  4187  4196  4209  4236  4237  4240  4246  4247  4356  4417
  4424  4425  5052  5061  5104  5110  5114  5141  5143  5156  5189  5206
  5207  5208  5227  5228  5328  5418  5489  5490  5546  5547  5548  5634
  5635  5637  5648  5668  5669  5671  6011  6232  6233  6239  6243  6249
  6258  6272  6273  6283  6288  6292  6312  7247  7255  7256  7259  7262
  7263  7274  7275  7279  7280  7289  7290  7292  7387  7397  7399  9578
  9777  9778  9845 10738 10740 10805 10842 10843 11307 11308 11354 11380
 11381 11410 11411 11419 11489 11491 11540 11545 11548 11550 11560 11561
 11562 11563 11564 11969 11970 12231 12232 12418 12419 13040 13041 13053
 13074 13142 13151 13152 13160 16270 16274 16634 16649 17206 17207 17208]
Sensor  cubierto

In [7]:
ini_date = '04/01/23 12:00:00'
end_date = '06/01/23 13:00:00'
target_folder="./time-data/evaluation/"

#[9:50, 11/10/2023] Aurora: setup: 20/12/2022 a las 15:00 a. m. hasta el 23/12/2022 a las 15:00 a. m.
#[9:50, 11/10/2023] Aurora: config: 04/01/2023 a las 12:00 a. m. hasta el 06/01/2023 a las 12:00 a. m.


t0 = (int)(datetime.strptime(ini_date, '%d/%m/%y %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0))

tN = (int)(datetime.strptime(end_date, '%d/%m/%y %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN))



nT=(int)(round((tN-t0-1)/time_step))
print(nT)


user_location={}
for tag in tags:
    user_location[tag]=[]
    series=read_csv(target_folder+tag+".tsv", sep="\t",parse_dates=True,header=None)
    user_location[tag]=  np.array(series.values[:,[1,2,3,4,5]])  
    print(user_location[tag].shape)
print(user_location["1"])


S=np.zeros((len(binary_sensors), nT))
for i,sensor in enumerate(binary_sensors):
    series=read_csv(target_folder+sensor+".tsv", sep="\t",parse_dates=True,header=None)
    S[i]=  np.array(series.values[:,[1]]).flatten()  
    print(S[i].shape)
print(S.shape)
print(np.where( (S[0]==1)))


04/01/23 12:00:00 Init date is 1672830000 day: 19361
06/01/23 13:00:00 End date is 1673006400 day: 19363
11760
(11760, 5)
(11760, 5)
[['None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None']
 ...
 ['None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None']]
(11760,)
(11760,)
(11760,)
(11760,)
(4, 11760)
(array([  212,   217,   337,   338,   375,   376,   382,   467,   468,
         492,   495,  1418,  2217,  2218,  2237,  2244,  2246,  2247,
        2250,  2335,  2336,  2823,  2844,  2845,  4771,  4819,  4869,
        4921,  4922,  5201,  5202,  5226,  5227,  6213,  6218,  6220,
        6243,  6244,  6287,  6288,  6289,  6301,  6336,  6337,  6384,
        6385,  6389,  6391,  6392,  6404,  6405,  6517,  6518,  6519,
        6523,  6525,  6553,  6554,  6555,  6559,  6560,  6574,  6578,
        7337,  7951,  7952,  7963,  7972,  7973,  8195,  8206,  8221,
        8222,  8516,  868

In [8]:
X_test={}
Y_test={}

for i,sensor in enumerate(binary_sensors):
    index_activation=np.where( (S[i]==1))[0]
    print("Sensor ",sensor, "#",i, "activation:",index_activation)
    
    X_test[sensor]=[]
    Y_test[sensor]=[]
    
    y_true={}
    gt=read_csv(target_folder+"gt-labelling-sensors/"+sensor+".tsv", sep="\t",parse_dates=True, na_values=0)
    gt=gt.fillna(0)
    for tx,tt in enumerate(gt["Ix"]):
        
        if(sum(gt.values[tx][2:])==0):
            maxI=0
        else:
            maxI=np.argmax(gt.values[tx][2:])+1
        y_true[tt]=(maxI)
        
        
    
    for t_activation in gt["Ix"]:
        t_activation=int(t_activation)
        t=absT(t_activation)
        #los datos de mi auro label (output) + coordenadas de 1 y 2 (input)
        #print("\t time_index:", t_activation, "LABEL:", y_true[t_activation], "in time", datetime.fromtimestamp(t).strftime("%Y-%m-%d %H:%M:%S"), "user 1:",user_location["1"][t_activation], "user 2:",user_location["2"][t_activation])
        
        user1_coords = [float(coord) if coord != 'None' else 0 for coord in user_location["1"][t_activation][:4]]
        user2_coords = [float(coord) if coord != 'None' else 0 for coord in user_location["2"][t_activation][:4]]
        label = y_true[t_activation]
        
        
        if label == 'None':
            label = 0

        x1, x2, y1, y2 = user1_coords
        x1_2, x2_2, y1_2, y2_2 = user2_coords
        X_test[sensor].append([x1, x2, y1, y2, x1_2, x2_2, y1_2, y2_2])

        Y_test[sensor].append(int(label)) 

    X_test[sensor] = np.array(X_test[sensor])
    Y_test[sensor] = np.array(Y_test[sensor])



Sensor  cubiertos # 0 activation: [  212   217   337   338   375   376   382   467   468   492   495  1418
  2217  2218  2237  2244  2246  2247  2250  2335  2336  2823  2844  2845
  4771  4819  4869  4921  4922  5201  5202  5226  5227  6213  6218  6220
  6243  6244  6287  6288  6289  6301  6336  6337  6384  6385  6389  6391
  6392  6404  6405  6517  6518  6519  6523  6525  6553  6554  6555  6559
  6560  6574  6578  7337  7951  7952  7963  7972  7973  8195  8206  8221
  8222  8516  8682 10845 10948 10949 11016 11017 11045 11046 11093 11273
 11284 11285 11510 11518 11545 11569 11589 11663 11667 11685]
Sensor  fridge # 1 activation: [  212   217   220   321   322   467  1388  1389  1393  2190  2191  2244
  2245  2336  2337  2342  2825  4817  4821  4822  4823  4824  4831  4832
  4923  5080  5081  5231  5233  5234  5776  5779  5780  5781  5796  5798
  6176  6177  6180  6181  6183  6186  6195  6196  6198  6202  6208  6209
  6212  6256  6261  6263  6266  6269  6272  6273  6274  6276  6277  62

In [9]:
print(X_train)
print(Y_train)

print(X_test)
print(Y_test)

{'cubiertos': array([[   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [4230.60060731, 4728.04685145, 4968.31894591, ..., 6636.87223834,
        1972.63104696, 7209.04487872],
       ...,
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ]]), 'fridge': array([[4463.56045862, 6932.39713202, 4466.99979007, 6933.99990217,
        4190.00746174, 7096.00342084, 4312.25321446, 7152.04703946],
       [4015.25735852, 6962.01855155, 4705.93129012, 7021.99322623,
        3897.46419052, 6888.99455962, 4015.83277329, 7783.42905112],
       [4250.190

In [10]:
total_y=[]
total_pred=[]
for i,sensor in enumerate(binary_sensors):
    print("Sensor:",sensor, "SVM")
    clf = svm.SVC()
    clf.fit(X_train[sensor], Y_train[sensor])
    Y_pred = clf.predict(X_test[sensor])
    print(classification_report(Y_test[sensor], Y_pred))
    total_y.extend(Y_test[sensor])
    total_pred.extend(Y_pred)
print("All sensors!", "SVM")    
print(classification_report(total_y, total_pred))

Sensor: cubiertos SVM
              precision    recall  f1-score   support

           1       0.06      0.02      0.03        41
           2       0.45      0.66      0.54        50

    accuracy                           0.37        91
   macro avg       0.25      0.34      0.29        91
weighted avg       0.27      0.37      0.31        91

Sensor: fridge SVM
              precision    recall  f1-score   support

           1       0.73      0.84      0.78        62
           2       0.63      0.47      0.54        36

    accuracy                           0.70        98
   macro avg       0.68      0.66      0.66        98
weighted avg       0.69      0.70      0.69        98

Sensor: microwave SVM
              precision    recall  f1-score   support

           1       0.75      1.00      0.86        21
           2       1.00      0.22      0.36         9

    accuracy                           0.77        30
   macro avg       0.88      0.61      0.61        30
weighted av

C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
from sklearn.ensemble import RandomForestClassifier
total_y=[]
total_pred=[]
for i,sensor in enumerate(binary_sensors):
    print("Sensor:",sensor, "RF")
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train[sensor], Y_train[sensor])
    Y_pred = clf.predict(X_test[sensor])
    print(classification_report(Y_test[sensor], Y_pred))
    total_y.extend(Y_test[sensor])
    total_pred.extend(Y_pred)
print("All sensors!", "RF")    
print("total_y",total_y)
print("total_pred",total_pred)
print(classification_report(total_y, total_pred))

Sensor: cubiertos RF
              precision    recall  f1-score   support

           1       0.27      0.34      0.30        41
           2       0.33      0.26      0.29        50

    accuracy                           0.30        91
   macro avg       0.30      0.30      0.30        91
weighted avg       0.30      0.30      0.30        91

Sensor: fridge RF
              precision    recall  f1-score   support

           1       0.77      0.58      0.66        62
           2       0.49      0.69      0.57        36

    accuracy                           0.62        98
   macro avg       0.63      0.64      0.62        98
weighted avg       0.66      0.62      0.63        98

Sensor: microwave RF
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.74      0.81      0.77        21
           2       0.75      0.33      0.46         9

    accuracy                           0.67        30
   macro avg  

C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\metrics\_classificat